In [1]:
def distance(time, speed):
    """Calculate the distance travelled at a constant speed for a known time."""
    return speed * time

def model_function(independent_data, parameter1, parameter2):
    """A template for the model function."""
    dependent_data = parameter1 * independent_data + parameter2
    return dependent_data

In [2]:
#control time, measure distance
import numpy
# derr is my estimate of errors measuring distance, my ruler is bad.
derr = 5 # metres
measured_times =numpy.arange(10,100,10) #time in seconds
measured_distances = numpy.array([ 108.2,  220.4,  360.2,  482.8,
        630.6,  793.9,  947.5, 1125.0, 1314.9]) # distance in metres
distance_errors = numpy.ones_like(measured_distances)*derr

# Original - bad


In [3]:

speeds = measured_distances / measured_times
average_speed = numpy.average(speeds)
print("Average speed is {:.04g} m/s".format(average_speed))

#Standard deviation of the data / sqrt(N) is NOT standard error!
mean_times_error = numpy.std(speeds, ddof=1)/numpy.sqrt(speeds.size)

#error on the average is fine, but it's not used!
mean_times_std = numpy.sqrt( numpy.mean( derr**2 * numpy.ones(speeds.size)) )

#error propagation, sum in quadrature, but I should be using the speed and speed errors!
speed_error = numpy.sqrt( numpy.mean( (distance_errors / measured_distances)**2) )* average_speed
print("Standard error in average speed is {:.03g} m/s".format(mean_times_error))
print("Error in average speed is {:.03g} m/s".format(speed_error))

Average speed is 12.66 m/s
Standard error in average speed is 0.436 m/s
Error in average speed is 0.236 m/s


# Ok - still wrong

In [4]:
speeds = measured_distances / measured_times

#error propagation for each point
speeds_sigma = speeds * (distance_errors/measured_distances)

#average speed
average_speed = numpy.mean(speeds)

#"average" standard deviation because each value is different. Ok, but not statistically correct
mean_times_error = numpy.mean(speeds_sigma)/numpy.sqrt(speeds.size)

#standard deviation, that's fine.
speed_error = numpy.std(speeds, ddof=1)
print("Average speed is {:.04g} m/s".format(average_speed))
print("Standard error in average speed is {:.03g} m/s".format(mean_times_error))
print("Standard deviation in average speed is {:.03g} m/s".format(speed_error))


Average speed is 12.66 m/s
Standard error in average speed is 0.0524 m/s
Standard deviation in average speed is 1.31 m/s


# Ok - still wrong
this would be okay if the distances were supposed to be the same.


In [5]:
#average distance and time
average_distance = numpy.mean(measured_distances)
average_time = numpy.mean(measured_times)

#correct standard error in distance
std_error_distance = derr/numpy.sqrt(measured_distances.size)

average_speed = average_distance/average_time

#error propagation of standard error
error_speed = average_speed * (std_error_distance/average_distance)

#sample standard deviation
std_distance = numpy.std(measured_distances, ddof=1)

#error propagation of sample standard deviation
error_speed_2 = average_speed * (std_distance/average_distance)

print("Average speed is {:.04g} m/s".format(average_speed))
print("Standard error in average speed is {:.03g} m/s".format(error_speed))

#I'm very big, because the "standard deviation" is not sensible for completely different distances.
print("Standard deviation in average speed is {:.03g} m/s".format(error_speed_2))


Average speed is 13.3 m/s
Standard error in average speed is 0.0333 m/s
Standard deviation in average speed is 8.26 m/s


# Correct - but complicated

In [6]:
#individual speeds
speeds = measured_distances / measured_times

#error propagation into speed
speeds_sigma = speeds * (distance_errors/measured_distances)

#average speed, weighted by the inverse square error (better measurements are higher weighted)
#v_bar = sum(v/s**2)/sum(1./s**2)

weighted_average_speed = numpy.sum(speeds*(1/speeds_sigma)**2)/numpy.sum((1/speeds_sigma)**2)

#standard error, 
#mean sigma is the added like the weighting above
#1./s_bar**2 = sum(1./s**2)
#MSE = s_bar / sqrt(N)
mean_sigma = 1./numpy.sqrt(numpy.mean(1./speeds_sigma**2))
mean_times_error = mean_sigma * 1./numpy.sqrt(speeds.size)

#standard deviation in speed, calculated from the correct weighted average, not the naive average.
speed_error = numpy.std(speeds - weighted_average_speed, ddof=1)
print("Average speed is {:.04g} m/s".format(weighted_average_speed))
print("Standard error in average speed is {:.03g} m/s".format(mean_times_error))
print("Standard deviation in average speed is {:.03g} m/s".format(speed_error))

Average speed is 13.66 m/s
Standard error in average speed is 0.0296 m/s
Standard deviation in average speed is 1.31 m/s


# Curve fit

In [7]:
from scipy.optimize import curve_fit
#curve fit..
popt, pcov = curve_fit(distance, measured_times, measured_distances,
                       absolute_sigma=True, sigma = distance_errors)
pvar = numpy.diag(pcov)

#done

print("Average speed is {:.04g} m/s".format(popt[0]))
print("Error in fitted speed is {:.03g} m/s".format(numpy.sqrt(pvar[0])))

Average speed is 13.66 m/s
Error in fitted speed is 0.0296 m/s
